In [1]:
import numpy as np
from sympy import *

In [11]:
#Signature is stored as tuple (m,i) where m is the monomial and i is the index
#Poly is stored as a polynomial ie. x**a + x**b +...
class LabeledPolynomial:
    def __init__(self, sign, poly, order = 'lex'):
        self.poly = poly
        self.sign = sign
        self.order = order
    
    def subtract(self,G):
        if signature_cmp(self.sign,G.sign,order = self.order):
            sign = self.sign
        else:
            sign = G.sign
        
        poly = self.poly-G.poly
        return(LabeledPolynomial(sign,poly,order = self.order))
    
    def multiply(self,m):
        sign = (self.sign[0]*m,self.sign[1])
        poly = expand(self.poly*m)
        return(LabeledPolynomial(sign,poly,order = self.order))

In [12]:
def monomial_cmp(u,v,order = 'lex'):
    #Return True if u > v
    temp = u+v
    highest_mon = LT(temp,order = order)
    if highest_mon == u:
        return True
    else:
        return False

    
def signature_cmp(f,g,order = 'lex'):
    #Return True if f > g
    if (f[1] < g[1]):
        return True
    if (f[1] == g[1]):
        if monomial_cmp(f[0],g[0]):
            return True
    return False

def CP(F,G):
    LM1 = LM(F.poly,order = F.order)
    LT1 = LT(F.poly,order = F.order)
    LM2 = LM(G.poly,order = G.order)
    LT2 = LT(G.poly,order = G.order)
    numerator = lcm(LM1,LM2)
    u = simplify(numerator/LT1)
    v = simplify(numerator/LT2)
    return((u,F,v,G))

def spoly(pair):
    lbp1 = pair[1].multiply(pair[0])
    lbp2 = pair[3].multiply(pair[2])
    return(lbp1.subtract(lbp2))

In [4]:
def is_divisible(F,B):
    for G in B:
        if G.sign[1] > F.sign[1]:
            lpp = LM(G.poly,order = G.order)
            if rem(F.sign[0],lpp) == 0:
                return True
    return False

def is_rewritable(F,B,u):
    index = B.index(F)
    for i in range(index+1,len(B)):
        G = B[i]
        if G.sign[1] == F.sign[1]:
            if G.sign[0] == 1:
                return True
            if not F.sign[0] == 1:
                lppg = LM(G.sign[0],order = G.order)
                lppf = LM(F.sign[0],order = F.order)
                lppf = lppf*u
                if rem(lppf,lppg) == 0:
                    return True
    return False
            

In [5]:
def f5_reduce(F,B):
    can_reduce = True
    while can_reduce:
        if (F.poly == 0):
            return(F)
        can_reduce = False
        for G in B:
            if rem(LM(F.poly,order = F.order),LM(G.poly,order = G.order)) == 0:
                m = LT(F.poly,order = F.order)/LT(G.poly,order = G.order)
                cxG = G.multiply(m)
                if signature_cmp(F.sign,cxG.sign):
                    if not is_divisible(cxG,B):
                        if not is_rewritable(G,B,m):
                            F = F.subtract(cxG)
                            can_reduce = True
                            break
                            
    return(F)

In [6]:
def f5_groebner(polys, order = 'lex'):
    B = []
    for i,p in enumerate(polys):
        b = LabeledPolynomial((1,i),p,order = order)
        B.append(b)
    n = len(B)
    pairs = []
    for i in range(n-1):
        for j in range(i+1,n):
            pairs.append(CP(B[i],B[j]))
    while len(pairs) > 0:
        cp = pairs.pop()
        uF = cp[1].multiply(cp[0])
        if not (is_divisible(uF,B) or is_rewritable(cp[1],B,cp[0])):
            vG = cp[3].multiply(cp[2])
            if not (is_divisible(vG,B) or is_rewritable(cp[3],B,cp[2])):
                S = spoly(cp)
                P = f5_reduce(S,B)
                if not P.poly == 0:
                    for b in B:
                        pairs.append(CP(b,P))
                    B.append(P)
    return([b.poly for b in B])

In [7]:
def reduce_groebner(G,order = 'lex'):
    H = []
    for i,g in enumerate(G):
        temp = [f for f in G if not f == g]
        m = LM(g,order = order)
        q,r = reduced(m,temp)
        if not r == 0:
            H.append(g/LC(g,order = order))
            
    for i,h in enumerate(H):
        temp = [f for f in H if not f == h]
        q,r = reduced(h,temp)
        H[i] = r
    return(H)

In [8]:
x,y,z = var('x,y,z')
f_list = [x**2+y,x*y-z,y**2+x*z]
sign_list = [(1,1),(1,2),(y,1)]

In [8]:
B = []
for i in range(len(f_list)):
    B.append(LabeledPolynomial(sign_list[i],f_list[i], order = 'grevlex'))


In [9]:
pair = CP(B[2],B[1])

In [10]:
uF = pair[1].multiply(pair[0])
vG = pair[3].multiply(pair[2])
print(uF.sign,uF.poly)
print(vG.sign,vG.poly)

(x*y, 1) x**2*z + x*y**2
(y, 2) x*y**2 - y*z


In [11]:
result = is_divisible(uF,B)
print(result)

True


In [12]:
x,y = var('x,y')
f_list = [x**2+x*y, x**2+y, x*y-y]
sign_list = [(1,1),(1,2),(1,1)]

In [13]:
B = []
for i in range(len(f_list)):
    B.append(LabeledPolynomial(sign_list[i],f_list[i], order = 'grevlex'))


In [14]:
pair = CP(B[0],B[2])

In [15]:
print(pair[0],pair[2])

y x


In [16]:
result = is_rewritable(pair[1],B,pair[0])
print(result)

True


In [17]:
x,y,z = var('x,y,z')
f_list = [x*z**2+y**2, x*y+x*z, y*z+z]
sign_list = [(1,1),(1,2),(1,3)]

In [18]:
B = []
for i in range(len(f_list)):
    B.append(LabeledPolynomial(sign_list[i],f_list[i], order = 'grevlex'))


In [19]:
pair = CP(B[1],B[2])
print(pair[0],pair[2])

z x


In [20]:
S = spoly(pair)

In [21]:
print(S.poly)
print(S.sign)

x*z**2 - x*z
(z, 2)


In [22]:
P = f5_reduce(S,B)

In [23]:
print(P.poly)

x*z**2 - x*z


In [8]:
x,y = var('x,y')
polys = [x*y-x, x**2-y]
G = groebner(polys,order = 'lex', method = 'f5b')
print(list(G))

[x**2 - y, x*y - x, y**2 - y]


In [9]:
test_G = f5_groebner(polys, order = 'lex')
print(test_G)

[x*y - x, x**2 - y, y**2 - y]


In [9]:
x,y = var('x,y')
polys = [x**3 - 2*x*y, x**2*y-2*y**2 + x,x**2 - x*y**2 + x**2*y**3]
G = groebner(polys,order = 'lex')
print(list(G))

[x - 2*y**2, y**3]


In [13]:
test_G = f5_groebner(polys,order = 'lex')

In [14]:
reduced_G = reduce_groebner(test_G,order = 'lex')

In [15]:
print(reduced_G)

[x - 2*y**2, y**3]


In [16]:
print(test_G)

[x**3 - 2*x*y, x**2*y + x - 2*y**2, x**2*y**3 + x**2 - x*y**2, -x**2 + 2*x*y**2 - 2*y**4, 2*x*y**5 + x*y**2 - 2*y**7 - 2*y**4, -x*y**4/2 - x*y/2 + y**9 + y**6 + y**3, -x*y**2/2 + 2*y**10 + y**7 + y**4, x*y + 4*y**12 - 2*y**3, -4*y**13 - 4*y**10 - 2*y**7, 2*x*y**3 + x - 2*y**5 - 2*y**2, x*y**2/2 + y**7, x/2 - 2*y**8 - y**5 - y**2, 4*y**10 + 4*y**7 + 2*y**4, -12*y**9 - 8*y**6 - 4*y**3, y**7/3 + y**4/6, y**6/6 + y**3/3, -3*y**4/2, 2*y**3]
